In [45]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import warnings
from tqdm import tqdm_notebook as tqdm

from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

sbn.set(style='white')
warnings.filterwarnings('ignore')

In [46]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [47]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def lemmatize(sentence):
    words = sentence.split()
    sentence = ""
    for word in words:
        sentence += lemmatizer.lemmatize(word) + " "
    sentence.strip()
    return sentence


def handle_contractions(sentence):
    cList = {"ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you you will",
    "you'll've": "you you will have",
    "you're": "you are",
    "you've": "you have"
    }
    for word, replacement in cList.items():
        sentence = sentence.replace(word, replacement)
    return sentence


def handle_punctuations(sentence):
    for punct in '?!.,#"$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’' + "&/-'":
        sentence = sentence.replace(punct, '')
    return sentence


def handle_stopwords(sentence):
    words = sentence.split()
    sentence = ""
    for word in words:
        if word not in stop_words:
            sentence += word + " "
    sentence.strip()
    return sentence


train['text'] = list(map(handle_contractions, tqdm(train['text'].tolist())))
train['text'] = list(map(handle_punctuations, tqdm(train['text'].tolist())))
train['text'] = list(map(handle_stopwords, tqdm(train['text'].tolist())))
train['text'] = list(map(lemmatize, tqdm(train['text'].tolist())))

test['text'] = list(map(handle_contractions, tqdm(test['text'].tolist())))
test['text'] = list(map(handle_punctuations, tqdm(test['text'].tolist())))
test['text'] = list(map(handle_stopwords, tqdm(test['text'].tolist())))
test['text'] = list(map(lemmatize, tqdm(test['text'].tolist())))

In [48]:
# x = train['text']
# y = train['sentiment']
# x_pred = test['text']

# vector = CountVectorizer().fit(x)
# x = vector.transform(x)
# x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# model = MultinomialNB()
# model.fit(x_train, y_train)
# y_pred = model.predict(x_val)
# score = f1_score(y_val, y_pred, average='micro')
# print(score)

0.6354166666666666


In [49]:
# x = train['text']
# y = train['sentiment']
# x_pred = test['text']

# vector = CountVectorizer().fit(x)
# x = vector.transform(x)
# x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# model = GaussianNB()
# model.fit(x_train.toarray(), y_train)
# y_pred = model.predict(x_val.toarray())
# score = f1_score(y_val, y_pred, average='micro')
# print(score)

0.6420454545454546


In [50]:
# x = train['text']
# y = train['sentiment']
# x_pred = test['text']

# vector = CountVectorizer().fit(x)
# x = vector.transform(x)
# x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# model = ComplementNB()
# model.fit(x_train, y_train)
# y_pred = model.predict(x_val)
# score = f1_score(y_val, y_pred, average='micro')
# print(score)

0.5369318181818182


In [51]:
x = train['text']
y = train['sentiment']
x_pred = test['text']

vector = CountVectorizer().fit(x)
x = vector.transform(x)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

model = BernoulliNB()
model.fit(x_train, y_train)
y_pred = model.predict(x_val)
score = f1_score(y_val, y_pred, average='micro')
print(score)

0.6931818181818182


In [52]:
confusion_matrix(y_val, y_pred)

array([[  7,   2, 102],
       [  2,   9, 164],
       [ 45,   9, 716]])

In [53]:
model = BernoulliNB().fit(x, y)
x_pred = vector.transform(x_pred)
y_pred = model.predict(x_pred)

sample = pd.read_csv('../input/sample.csv')
sample['sentiment'] = y_pred
sample.to_csv('BernoulliNB-with-Preprocessing.csv', index=False)